# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [3]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


In [4]:
df.shape

(397, 4)

## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables relative to whether someone got admitted or not. Think in terms of for a certain prestige level, how many people got admitted and didnt get admitted

In [112]:
# frequency table for prestige and whether or not someone was admitted

f_table = pd.crosstab(index=df["admit"], columns=df["prestige"])

f_table.index = ["not admitted", "admitted"]

f_table

prestige,1.0,2.0,3.0,4.0
not admitted,28,97,93,55
admitted,33,53,28,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [64]:
dummy_ranks = pd.get_dummies(df_raw["prestige"])

dummy_ranks.head()

,1.0,2.0,3.0,4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: if my class variable is K, so we need K-1 class variables

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [69]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.iloc[:, 0:])
print handCalc.head()

   admit    gre   gpa  1.0  2.0  3.0  4.0
0      0  380.0  3.61    0    0    1    0
1      1  660.0  3.67    0    0    1    0
2      1  800.0  4.00    1    0    0    0
3      1  640.0  3.19    0    0    0    1
4      0  520.0  2.93    0    0    0    1


In [90]:
# crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted

f_prestige_1 = pd.crosstab(index=handCalc["admit"], columns=handCalc[1.0])

f_prestige_1

1.0,0,1
admit,,
0,245,28
1,94,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [109]:
odds_event_1 = 33./94

odds_event_1

0.35106382978723405

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [108]:
odds_event_2 = 94./33

odds_event_2

2.8484848484848486

#### 3.3 Calculate the odds ratio

In [115]:
odds_ratio_1 = odds_event_2 / odds_event_1

print odds_ratio_1

8.11386593205


#### 3.4 Write this finding in a sentenance: 

Answer: The odds of being admitted to grad school if you not attended a #1 ranked college are higher then the odds of being admitted if you attended one #1 ranked college.

#### 3.5 Print the cross tab for prestige_4

In [114]:
f_prestige_4 = pd.crosstab(index=handCalc["admit"], columns=handCalc[4.0])

f_prestige_4

4.0,0,1
admit,,
0,218,55
1,115,12


#### 3.6 Calculate the OR 

In [116]:
odds_event_3 = 12./115

print odds_event_3

odds_event_4 = 115./12

print odds_event_4

odds_ratio_2 = odds_event_4 / odds_event_3

print odds_ratio

0.104347826087
9.58333333333
8.11386593205


#### 3.7 Write this finding in a sentence

Answer: The odds ratio for this case is the same as the odds ratio for #1 ranked college.

## Part 4. Analysis

In [183]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.iloc[:,0:3]).dropna()
print data.head()

   admit    gre   gpa  1.0  2.0  3.0
0      0  380.0  3.61    0    0    1
1      1  660.0  3.67    0    0    1
2      1  800.0  4.00    1    0    0
3      1  640.0  3.19    0    0    0
4      0  520.0  2.93    0    0    0


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [184]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [214]:
train_cols = data.iloc[:,1:7]

#### 4.2 Fit the model

In [215]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

lm = LogisticRegression()

lm.fit(train_cols, data["admit"])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

#### 4.3 Print the summary results

In [216]:
print lm.coef_
print lm.intercept_
print data.admit.mean()

[[ 0.00182563  0.26553082  1.22528104  0.56898046  0.01121686 -1.58458246]]
[-1.58458246]
0.317380352645


#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [217]:
print np.exp(lm.coef_)

[[ 1.0018273   1.30412304  3.40512293  1.76646516  1.01128001  0.20503339]]


#### 4.5 Interpret the OR of Prestige_2

Answer: Prestige = 2 has the higher odds. Also, schools prestige as it approaches 4 decreases the odds of getting admitted.

#### 4.6 Interpret the OR of GPA

Answer: GPA is variable that impact the most the admission.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [180]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [181]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [ ]:
# recreate the dummy variables

# keep only what we need for making predictions


#### 5.2 Make predictions on the enumerated dataset

#### 5.3 Interpret findings for the last 4 observations

Answer: 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.